In [132]:
import os
import sys
import pandas as pd
from importlib import reload
import time

In [2]:
import enviserv.dictan
from enviserv.dictan import DictAnalyzer
import pandserv as pds # сервисные функции для пандас и не только

In [3]:
from sparkserv import SparkApp, Cols

In [4]:
import sparkserv

In [5]:
spark_app = SparkApp(my_logger_create_level = 'INFO')

INFO:spark_app:spark_master_ip: 172.18.0.2
INFO:spark_app:starting building spark app object: pyspark-taxi-forecasting
INFO:spark_app:built spark app object: <pyspark.sql.session.SparkSession object at 0x7fd17e422810>


In [5]:
spark_master_ip = spark_app.get_spark_master_ip()
# print(spark_master_ip)

INFO:spark_app:spark_master_ip: 172.18.0.2


In [6]:
# spark = spark_app.build_spark_app(spark_master_ip=spark_master_ip)
spark = spark_app.spark

In [7]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2022.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 29.7 ms, sys: 9.12 ms, total: 38.8 ms
Wall time: 44.8 s


In [8]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

In [9]:
pd.set_option('display.max_colwidth', None)  # Показать полное содержимое ячеек столбцов

In [109]:
# создадим объект для представления имен полей
ct = sparkserv.Cols(taxi)

ct.get_aliases()

{'trip_i': 'Trip ID',
 'taxi_i': 'Taxi ID',
 'trip_st': 'Trip Start Timestamp',
 'trip_et': 'Trip End Timestamp',
 'trip_s': 'Trip Seconds',
 'trip_m': 'Trip Miles',
 'pickup_ct': 'Pickup Census Tract',
 'dropoff_ct': 'Dropoff Census Tract',
 'pickup_ca': 'Pickup Community Area',
 'dropoff_ca': 'Dropoff Community Area',
 'fare': 'Fare',
 'tips': 'Tips',
 'tolls': 'Tolls',
 'extras': 'Extras',
 'trip_t': 'Trip Total',
 'payment_t': 'Payment Type',
 'company': 'Company',
 'pickup_cl': 'Pickup Centroid Latitude',
 'pickup_clon': 'Pickup Centroid Longitude',
 'pickup_cloc': 'Pickup Centroid Location',
 'dropoff_cl': 'Dropoff Centroid Latitude',
 'dropoff_clon': 'Dropoff Centroid Longitude',
 'dropoff_cloc': 'Dropoff Centroid  Location'}

In [10]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';',index_col='Column Name')

table_descr

Description  \
Column Name                                                                                                                                                                                                                 
Trip ID                                                                                                                                                                                 A unique identifier for the trip.   
Taxi ID                                                                                                                                                                                 A unique identifier for the taxi.   
Trip Start Timestamp                                                                                                                                             When the trip started rounded to the nearest 15 minutes.   
Trip End Timestamp                                                                                                                                                 When the trip ended rounded to the nearest 15 minutes.   
Trip Seconds                                                                                                                                                                                 Time of the trip in seconds.   
Trip Miles                                                                                                                                                                                 Distance of the trip in miles.   
Pickup Census Tract                                     The Census Tract where the trip began. For privacy  this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.   
Dropoff Census Tract                                    The Census Tract where the trip ended. For privacy  this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.   
Pickup Community Area                                                                                                   The Community Area where the trip began. This column will be blank for locations outside Chicago.   
Dropoff Community Area                                                                                                  The Community Area where the trip ended. This column will be blank for locations outside Chicago.   
Fare                                                                                                                                                                                               The fare for the trip.   
Tips                                                                                                                                                      The tip for the trip. Cash tips generally will not be recorded.   
Tolls                                                                                                                                                                                             The tolls for the trip.   
Extras                                                                                                                                                                                        Extra charges for the trip.   
Trip Total                                                                                                                                                      Total cost of the trip the total of the previous columns.   
Payment Type                                                                                                                                                                                Type of payment for the trip.   
Company                                                                                                                                                                                                 The taxi company.   
Pickup 

In [11]:
pd.reset_option('display.max_colwidth')  # Сброс максимальной ширины столбцов к значению по умолчанию

In [10]:
# %%time
# describe_result = taxi.describe().toPandas()

# time cost i5 (7g), 4 workers (x) 2gb RAM 
# CPU times: user 352 ms, sys: 32.2 ms, total: 384 ms
# Wall time: 2min 7s

CPU times: user 352 ms, sys: 32.2 ms, total: 384 ms
Wall time: 2min 7s


In [11]:
# output_file_path = '/work/data/2022_describe.csv'
# describe_result.to_csv(output_file_path, index=False) #transpose().

In [25]:
dr = pd.read_csv('/work/data/2022_describe.csv',sep=',',index_col='summary')

In [26]:
drf = dr.copy()
pds.get_df_formated(drf, '`',2, 10)

In [27]:
drf.transpose()

summary                         count               mean      stddev  \
Trip ID                     6`382`425                nan         nan   
Taxi ID                     6`382`425                nan         nan   
Trip Start Timestamp        6`382`425                nan         nan   
Trip End Timestamp          6`382`213                nan         nan   
Trip Seconds                6`380`960           1`198.21    1`895.66   
Trip Miles                  6`382`369               6.19        8.00   
Pickup Census Tract         2`623`831  17`031`468`160.38  368`945.90   
Dropoff Census Tract        2`675`331  17`031`411`846.86  345`773.49   
Pickup Community Area       5`868`572              32.35       25.20   
Dropoff Community Area      5`748`741              25.84       20.93   
Fare                        6`378`889              21.73       49.42   
Tips                        6`378`889               2.75        4.08   
Tolls                       6`378`889               0.02        7.66   
Extras                      6`378`889               2.16       21.75   
Trip Total                  6`378`889              26.83       56.96   
Payment Type                6`382`425                nan         nan   
Company                     6`382`425                nan         nan   
Pickup Centroid Latitude    5`870`874              41.90        0.06   
Pickup Centroid Longitude   5`870`874             -87.69        0.10   
Pickup Centroid Location    5`870`874                nan         nan   
Dropoff Centroid Latitude   5`784`494              41.89        0.06   
Dropoff Centroid Longitude  5`784`494             -87.66        0.07   
Dropoff Centroid  Location  5`784`494                nan         nan   

summary                                min             max  
Trip ID                         000000bb18      ffffff1aae  
Taxi ID                         0041f8f0c9      fff84aa08a  
Trip Start Timestamp            01/01/2022      12/31/2022  
Trip End Timestamp              01/01/2022      12/31/2022  
Trip Seconds                             0          86`341  
Trip Miles                               0        2`967.54  
Pickup Census Tract         17`031`010`100  17`031`980`100  
Dropoff Census Tract        17`031`010`100  17`031`980`100  
Pickup Community Area                    1              77  
Dropoff Community Area                   1              77  
Fare                                     0        9`999.75  
Tips                                     0             496  
Tolls                                    0        6`666.66  
Extras                                   0        8`888.88  
Trip Total                               0        9`999.75  
Payment Type                          Cash         Unknown  
Company                         24 Seven T       U Taxicab  
Pickup Centroid Latitude             41.65           42.02  
Pickup Centroid Longitude           -87.91          -87.53  
Pickup Centroid Location        POINT (-87      POINT (-87  
Dropoff Centroid Latitude            41.66           42.02  
Dropoff Centroid Longitude          -87.91          -87.53  
Dropoff Centroid  Location      POINT (-87      POINT (-87

In [28]:
val = int(dr.transpose()['count'].astype(float).max())

In [29]:
print(pds.gvf(val))

6'382'425


In [18]:
%%time
row_count = taxi.count()

# time cost i5 (7g), 4 workers (x) 2gb RAM
# CPU times: user 5.85 ms, sys: 871 µs, total: 6.72 ms
# Wall time: 11.8 s

CPU times: user 5.85 ms, sys: 871 µs, total: 6.72 ms
Wall time: 11.8 s


In [19]:
print(pds.gvf(row_count))

6'382'425


In [74]:
# способ преобразования object столбца и
# фильтрации по стоблцу с использование to_numeric
drt = dr.transpose()
drtf = drt[pd.to_numeric(drt['count'], errors='coerce') > row_count*0.99][['count']]
# отображение отформатированных данных
pds.get_df_formated(drtf,'`',2, 10)
print('==================',pds.gvf(row_count))
drtf

================== 6'382'425


summary                   count
Trip ID               6`382`425
Taxi ID               6`382`425
Trip Start Timestamp  6`382`425
Trip End Timestamp    6`382`213
Trip Seconds          6`380`960
Trip Miles            6`382`369
Fare                  6`378`889
Tips                  6`378`889
Tolls                 6`378`889
Extras                6`378`889
Trip Total            6`378`889
Payment Type          6`382`425
Company               6`382`425

In [104]:
# способ преобразования object столбца и
# фильтрации по стоблцу с использование astype
# 
# можно сразу применять тип и фильтровать, но значения в поле не поменяют тип
drtf1 = dr.transpose()[dr.transpose()['count'].astype(int) > -1][['count']]
# 
# если нужны вычисления с полем, то надо создать дф
drt1 = dr.transpose()
drt1['count'] = drt1['count'].astype(float)
drt1['not_null']=drt1['count']/row_count
# drt1[['count','not_null']]

In [108]:
drtf1 = drt1[drt1['count'] > -1][['count','not_null']]
# display(drtf1)
# отображение отформатированных данных
pds.get_df_formated(drtf1,'`',2, 10)
display(drtf1)
print('================================')
print('==total rows count==',pds.gvf(row_count))
print('================================')
print('=== columns count======',int(len(drtf1)))
print('================================')

summary                         count not_null
Trip ID                     6`382`425        1
Taxi ID                     6`382`425        1
Trip Start Timestamp        6`382`425        1
Trip End Timestamp          6`382`213     1.00
Trip Seconds                6`380`960     1.00
Trip Miles                  6`382`369     1.00
Pickup Census Tract         2`623`831     0.41
Dropoff Census Tract        2`675`331     0.42
Pickup Community Area       5`868`572     0.92
Dropoff Community Area      5`748`741     0.90
Fare                        6`378`889     1.00
Tips                        6`378`889     1.00
Tolls                       6`378`889     1.00
Extras                      6`378`889     1.00
Trip Total                  6`378`889     1.00
Payment Type                6`382`425        1
Company                     6`382`425        1
Pickup Centroid Latitude    5`870`874     0.92
Pickup Centroid Longitude   5`870`874     0.92
Pickup Centroid Location    5`870`874     0.92
Dropoff Centroid Latitude   5`784`494     0.91
Dropoff Centroid Longitude  5`784`494     0.91
Dropoff Centroid  Location  5`784`494     0.91

==total rows count== 6'382'425
=== columns count====== 23


Поля "Census Tract" и "Community Area" представляют собой два различных уровня административных единиц:

**Census Tract:**  
Это статистические единицы, используемые в переписи населения в США, проводимой каждые 10 лет.
Census Tract обычно представляют собой географические области, населенные от 1 200 до 8 000 человек.
Они предназначены для обеспечения стандартизированных данных для анализа населения, доходов, расового и этнического состава и других социально-экономических параметров.

Далее для краткости буду называть данную адм.единицу - **Квартал**

**Community Area:**  
Это административные районы в городе Чикаго, разработанные городским правительством.
Community Areas используются для организации и управления городскими службами и ресурсами.
Обычно Community Areas выступают в качестве единиц анализа для изучения социальных и экономических показателей, таких как уровень преступности, доступность образования и здравоохранения и др.

Далее для краткости буду называть данную адм.единицу - **Район**

Community Area представляют собой административные районы, охватывающие большие части города, в то время как Census Tract более мелкие географические области, которые могут быть распределены по всему Community Area. Таким образом, Community Area охватывает несколько Census Tract и представляет собой более крупную административную единицу.

В идеальной ситуации каждый Census Tract должен быть частью только одной Community Area, чтобы избежать перекрытий и дублирования данных. Однако в реальности могут быть исключения из-за изменений границ административных районов или других факторов.

#### **Пропуски**  
Видим, что  
- 5 полей из 23 не содержат пропусков. 8 полей заполнены более чем на 99%.  
- `Pickup Census Tract`, `Dropoff Census Tract` заполнены менее, чем на 50%
- соответствующие поля с геоданными заполнены более, чем на 90%

Поскольку прогноз строится для квартала, следует связать поля геоданных и кварталов. Попробую заполнить пропуски. В случае невозможности заполненения пропусков для кварталов, в качестве ключа-аггрегатора буду использовать геоточки.

In [110]:
# Функция для вычисления частот
def frq(df, column):
    # Вычисляем частоту для каждого значения
    freq_df = df.groupBy(column).count().orderBy("count", ascending=False)
    return freq_df

In [112]:
%%time
pct_frq = frq(taxi,ct.pickup_ct)
pct_frq.show()

+-------------------+-------+
|Pickup Census Tract|  count|
+-------------------+-------+
|               null|3758594|
|        17031980000| 519802|
|        17031320100| 304178|
|        17031839100| 284131|
|        17031281900| 188178|
|        17031081500| 174424|
|        17031081403| 112656|
|        17031330100| 108691|
|        17031081401| 104198|
|        17031980100|  85226|
|        17031081201|  84359|
|        17031081300|  76482|
|        17031320400|  71039|
|        17031081700|  63163|
|        17031320600|  61120|
|        17031280100|  55753|
|        17031841000|  41201|
|        17031081800|  39526|
|        17031081402|  35200|
|        17031081600|  30245|
+-------------------+-------+
only showing top 20 rows

CPU times: user 8.5 ms, sys: 10.4 ms, total: 18.9 ms
Wall time: 19.5 s


In [113]:
%%time
dct_frq = frq(taxi,ct.dropoff_ct)
dct_frq.show()

+--------------------+-------+
|Dropoff Census Tract|  count|
+--------------------+-------+
|                null|3707094|
|         17031839100| 328855|
|         17031320100| 277879|
|         17031980000| 243652|
|         17031081500| 160251|
|         17031330100| 139399|
|         17031081403| 133973|
|         17031281900| 129117|
|         17031081700| 114125|
|         17031081401| 100596|
|         17031081201|  87949|
|         17031833000|  83912|
|         17031320400|  81112|
|         17031081800|  76118|
|         17031081300|  75221|
|         17031280100|  67209|
|         17031320600|  59928|
|         17031980100|  48559|
|         17031841000|  42231|
|         17031081600|  40664|
+--------------------+-------+
only showing top 20 rows

CPU times: user 13.4 ms, sys: 1.9 ms, total: 15.3 ms
Wall time: 14.8 s


In [114]:
%%time
pca_frq = frq(taxi,ct.pickup_ca)
pca_frq.show()

+---------------------+-------+
|Pickup Community Area|  count|
+---------------------+-------+
|                    8|1482215|
|                   32|1031010|
|                   76| 989907|
|                   28| 531280|
|                 null| 513853|
|                   33| 229646|
|                    6| 229347|
|                   56| 183449|
|                    7| 150637|
|                    3| 100679|
|                   77|  76036|
|                   24|  75849|
|                    2|  47266|
|                   41|  44448|
|                   35|  35625|
|                    1|  35251|
|                   43|  33720|
|                   38|  33032|
|                   22|  31390|
|                   16|  30553|
+---------------------+-------+
only showing top 20 rows

CPU times: user 15.6 ms, sys: 9.18 ms, total: 24.8 ms
Wall time: 14.9 s


In [115]:
def compare_unique_values(df, field1, field2):
    # Извлекаем уникальные значения для каждого поля
    unique_values_field1 = df.select(field1).distinct().collect()
    unique_values_field2 = df.select(field2).distinct().collect()
    
    # Преобразуем результаты в множества
    set_field1 = {row[field1] for row in unique_values_field1}
    set_field2 = {row[field2] for row in unique_values_field2}
    
    # Находим значения, которые есть в set_field1, но нет в set_field2
    only_in_field1 = {value for value in set_field1 if value not in set_field2}
    
    # Находим значения, которые есть в set_field2, но нет в set_field1
    only_in_field2 = {value for value in set_field2 if value not in set_field1}
    
    return only_in_field1, only_in_field2, set_field1, set_field2

In [116]:
%%time
only_pickup, only_dropoff, pickup, dropoff = \
compare_unique_values(taxi, ct.pickup_ct,ct.dropoff_ct)

CPU times: user 55.2 ms, sys: 11 ms, total: 66.2 ms
Wall time: 29 s


In [117]:
print('`Pickup Census Tract` distinct count:',len(pickup), ' | only in pickup set:',len(only_pickup))
print('`Dropoff Census Tract` distinct count:',len(dropoff), ' | only in dropoff set:',len(only_dropoff))

`Pickup Census Tract` distinct count: 666  | only in pickup set: 34
`Dropoff Census Tract` distinct count: 812  | only in dropoff set: 180


### Тренировка отбора тэйком

In [118]:
%%time
taxi.take(1)

CPU times: user 6.64 ms, sys: 945 µs, total: 7.58 ms
Wall time: 297 ms


[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None)]

In [119]:
taxi.take(3)[:2]

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12', Taxi ID='449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='

In [139]:
%%time
# Отбор значений для одного элемента объекта Row (одно поле)
[row[ct.trip_i] for row in taxi.take(3)[:2]]

CPU times: user 5.74 ms, sys: 642 µs, total: 6.38 ms
Wall time: 203 ms


['bcfa19f2539021c054809d4c3993d226996ae095',
 '2aba69ff015f9ea8e7bff43cab7eddb228f34a12']

In [140]:
# список словарей
[{field: row[field] for field in [ct.trip_i, ct.taxi_i]} for row in taxi.take(3)[:2]]

[{'Trip ID': 'bcfa19f2539021c054809d4c3993d226996ae095',
  'Taxi ID': '368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334'},
 {'Trip ID': '2aba69ff015f9ea8e7bff43cab7eddb228f34a12',
  'Taxi ID': '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317'}]

In [141]:
# список списков для нескольких полей, несколько проходов
[[row[ct.trip_i] for row in taxi.take(3)[:2]], [row[ct.taxi_i] for row in taxi.take(3)[:2]]]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

In [142]:
# список кортежей, один проход
list(zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in taxi.take(3)[:2]]))

[('bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 ('368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317')]

In [143]:
# список списков - один проход
[list(t) for t in zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in taxi.take(3)[:2]])]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

In [144]:
# список списков - произвольный список полей один проход.
fields = [ct.trip_i, ct.taxi_i]

[list(t) for t in zip(*[[row[field] for field in fields] for row in taxi.take(3)[:2]])]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

***Использование `zip(*...)` и генератора списка*** для преобразования кортежей в списки имеет минимальные накладные расходы. Эти операции довольно быстры, так как они выполняются в памяти и не требуют значительных вычислительных ресурсов.

Вот краткий обзор того, как работают эти преобразования:

**zip(*...)**: Это операция, которая транспонирует список списков. Она создает итератор, который группирует элементы из каждого из входных списков. Это делается за один проход по данным, что делает его очень эффективным.
Генератор списка: Преобразование каждого кортежа в список с помощью генератора списка `[list(t) for t in ...]` также выполняется за один проход по данным и выполняет простое преобразование типов. Это эффективно и не требует значительных вычислительных затрат.  
Для небольших наборов данных (несколько десятков или сотен элементов) накладные расходы будут практически незаметными. Для больших наборов данных, таких как миллионы элементов, накладные расходы могут стать более заметными, но по-прежнему будут минимальными по сравнению с другими более затратными операциями (например, ввод-вывод на диск или сложные вычисления).

Чтобы продемонстрировать минимальные накладные расходы, давайте рассмотрим пример с измерением времени:

In [145]:
# Пример данных
rows = [{ct.trip_i: f'trip_id_{i}', ct.taxi_i: f'taxi_id_{i}'} for i in range(1000000)]

# Измеряем время выполнения преобразования
start_time = time.time()
[list(t) for t in zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in rows])]
end_time = time.time()

print("Время выполнения: ", end_time - start_time)

Время выполнения:  3.4812140464782715


В большинстве случаев такие преобразования в Python очень эффективны, и вам не стоит беспокоиться о них, если нет необходимости оптимизировать производительность до предела.

In [191]:
%%time
taxi.select(ct.trip_i).take(3)[:2]

CPU times: user 0 ns, sys: 10.2 ms, total: 10.2 ms
Wall time: 118 ms


[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095'),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12')]

In [30]:
%%time
pck = taxi.select('Pickup Census Tract').distinct().collect()

CPU times: user 11.8 ms, sys: 8.17 ms, total: 20 ms
Wall time: 14.3 s


In [31]:
%%time
taxi.createOrReplaceTempView("taxis")
result = spark.sql("SELECT DISTINCT `Pickup Census Tract` FROM taxis")
pck2 = result.collect()

CPU times: user 14.6 ms, sys: 1.8 ms, total: 16.4 ms
Wall time: 15.3 s


In [32]:
import time
from tqdm import tqdm

In [33]:
# Количество испытаний
num_trials = 5

# Время выполнения для первого блока кода: сбор уникальных значений с помощью DataFrame API
pck_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    pck = taxi.select('Pickup Census Tract').distinct().collect()
    end_time = time.time()
    pck_times.append(end_time - start_time)

# Вывод среднего времени выполнения для первого блока кода
print("Среднее время выполнения для первого блока кода: {:.2f} секунд".format(sum(pck_times) / num_trials))

100%|██████████| 5/5 [01:09<00:00, 13.84s/it]

Среднее время выполнения для первого блока кода: 13.83 секунд


In [34]:
# Время выполнения для второго блока кода: сбор уникальных значений с помощью SQL запроса
result_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    taxi.createOrReplaceTempView("taxis")
    result = spark.sql("SELECT DISTINCT `Pickup Census Tract` FROM taxis")
    pck2 = result.collect()
    end_time = time.time()
    result_times.append(end_time - start_time)

# Вывод среднего времени выполнения для второго блока кода
print("Среднее время выполнения для второго блока кода: {:.2f} секунд".format(sum(result_times) / num_trials))


100%|██████████| 5/5 [01:09<00:00, 13.93s/it]

Среднее время выполнения для второго блока кода: 13.93 секунд


In [174]:
%%time
f1 = ct.pickup_ct # 'Pickup Census Tract'
f2 = ct.pickup_cl # 'Pickup Centroid Latitude'
f3 = ct.pickup_clon # 'Pickup Centroid Longitude'
pckgeo = taxi.select(f1, f2, f3).distinct().collect()

CPU times: user 28.5 ms, sys: 4.2 ms, total: 32.7 ms
Wall time: 17.2 s


In [175]:
len(pckgeo)

743

In [39]:
pckgeo[:3]

[Row(Pickup Census Tract=17031071300, Pickup Centroid Location='POINT (-87.6414597588 41.9217783562)', Pickup Centroid Location='POINT (-87.6414597588 41.9217783562)'),
 Row(Pickup Census Tract=None, Pickup Centroid Location='POINT (-87.6487879519 41.8361501547)', Pickup Centroid Location='POINT (-87.6487879519 41.8361501547)'),
 Row(Pickup Census Tract=17031040100, Pickup Centroid Location='POINT (-87.681993542 41.9833308063)', Pickup Centroid Location='POINT (-87.681993542 41.9833308063)')]

In [40]:
len([row for row in pckgeo if row[f1] is None])

78

In [41]:
from collections import defaultdict

In [42]:
# Инициализация словаря для хранения значений
tract_to_coords = defaultdict(list)

# Заполнение словаря
for row in tqdm(pckgeo):
    tract = row[f1]
    coords = (row[f3], row[f2])
    tract_to_coords[tract].append(coords)

# Поиск районов с разными значениями широты и долготы
repeated_tracts = {tract: coords for tract, coords in tract_to_coords.items() if len(set(coords)) > 1}

100%|██████████| 743/743 [00:00<00:00, 56315.15it/s]


In [44]:
# Вывод результатов
for tract, coords in repeated_tracts.items():
    print(f"Район: {tract}, Количетво геоточек: {len(coords)}")
    print()

# Количество таких районов
print(f"Количество районов с разными значениями широты и долготы: {len(repeated_tracts)}")

Район: None, Количетво геоточек: 78

Количество районов с разными значениями широты и долготы: 1


In [45]:
for geop in repeated_tracts[None]:
    if geop == (None,None): continue
    # print(f"geop {geop}")
    for tract, geos in tract_to_coords.items():
        if tract is None: continue
        if geop in geos:
            print(f"geopoint: {geop} - {tract}")

In [71]:
# tract_to_coords

In [45]:
%%time
geos_pck = taxi.select(f2, f3).distinct().orderBy(f2, f3).collect()

CPU times: user 24.6 ms, sys: 9.45 ms, total: 34 ms
Wall time: 14.4 s


In [46]:
len(geos_pck)

616

In [47]:
geos_pck[:5]

[Row(Pickup Centroid Latitude=None, Pickup Centroid Longitude=None),
 Row(Pickup Centroid Latitude=41.651921576, Pickup Centroid Longitude=-87.564929171),
 Row(Pickup Centroid Latitude=41.660136051, Pickup Centroid Longitude=-87.60284764),
 Row(Pickup Centroid Latitude=41.663670652, Pickup Centroid Longitude=-87.540935513),
 Row(Pickup Centroid Latitude=41.663759731, Pickup Centroid Longitude=-87.637304085)]

In [48]:
%%time
f2 = 'Pickup Centroid Latitude'
f3 = 'Pickup Centroid Longitude'
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNotNull() &
                taxi[f3].isNotNull()).distinct().orderBy(taxi[f2].asc(), taxi[f3].asc()).collect()

CPU times: user 38.4 ms, sys: 2.33 ms, total: 40.7 ms
Wall time: 14.6 s


In [49]:
geos_pck_not_null[:5]

[Row(Pickup Centroid Latitude=41.651921576, Pickup Centroid Longitude=-87.564929171),
 Row(Pickup Centroid Latitude=41.660136051, Pickup Centroid Longitude=-87.60284764),
 Row(Pickup Centroid Latitude=41.663670652, Pickup Centroid Longitude=-87.540935513),
 Row(Pickup Centroid Latitude=41.663759731, Pickup Centroid Longitude=-87.637304085),
 Row(Pickup Centroid Latitude=41.665167734, Pickup Centroid Longitude=-87.535284961)]

In [50]:
%%time
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNull() &
                taxi[f3].isNull()).collect()

CPU times: user 3.96 s, sys: 92.4 ms, total: 4.05 s
Wall time: 17.9 s


In [51]:
%%time
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNull() & taxi[f3].isNull())

CPU times: user 273 ms, sys: 984 µs, total: 274 ms
Wall time: 295 ms


In [53]:
%%time
print(pds.gvf(geos_pck_not_null.count()))

511'551
CPU times: user 0 ns, sys: 7.49 ms, total: 7.49 ms
Wall time: 13.1 s


In [55]:
%%time
taxi.select(f2, f3).filter(taxi[f2].isNull() &
                taxi[f3].isNotNull()).count()

CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 12.9 s


0

In [56]:
%%time
taxi.select(f2, f3).filter(taxi[f2].isNotNull() &
                taxi[f3].isNull()).count()

CPU times: user 6.02 ms, sys: 8.44 ms, total: 14.5 ms
Wall time: 13.5 s


0

In [93]:
taxi.select(ct.pickup_ca).distinct().count()

78

In [94]:
taxi.select(ct.dropoff_ca).distinct().count()

78

In [95]:
ct.testtt()

*******-------------*******


In [96]:
ct.get_aliases()

{'trip_i': 'Trip ID',
 'taxi_i': 'Taxi ID',
 'trip_st': 'Trip Start Timestamp',
 'trip_et': 'Trip End Timestamp',
 'trip_s': 'Trip Seconds',
 'trip_m': 'Trip Miles',
 'pickup_ct': 'Pickup Census Tract',
 'dropoff_ct': 'Dropoff Census Tract',
 'pickup_ca': 'Pickup Community Area',
 'dropoff_ca': 'Dropoff Community Area',
 'fare': 'Fare',
 'tips': 'Tips',
 'tolls': 'Tolls',
 'extras': 'Extras',
 'trip_t': 'Trip Total',
 'payment_t': 'Payment Type',
 'company': 'Company',
 'pickup_cl': 'Pickup Centroid Latitude',
 'pickup_clon': 'Pickup Centroid Longitude',
 'pickup_cloc': 'Pickup Centroid Location',
 'dropoff_cl': 'Dropoff Centroid Latitude',
 'dropoff_clon': 'Dropoff Centroid Longitude',
 'dropoff_cloc': 'Dropoff Centroid  Location'}

In [98]:
taxi.take(1)

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None)]

In [145]:
taxi.filter(taxi[ct.trip_m]>10).count()

1777033

In [123]:
long = taxi.filter((taxi[ct.trip_m]>9.9) & (taxi[ct.trip_m]<10.1) 
                   & taxi[ct.pickup_cl].isNotNull() & taxi[ct.dropoff_cl].isNotNull() 
                   & taxi[ct.pickup_clon].isNotNull() & taxi[ct.dropoff_clon].isNotNull()
                  )

In [124]:
long.count()

20855

In [140]:
long = long.withColumn('Distance',(
                                ((taxi[ct.pickup_cl]-taxi[ct.dropoff_cl])**2 +
                                (taxi[ct.pickup_clon]-taxi[ct.dropoff_clon])**2))**0.5)

In [146]:
%%time
long.select(ct.pickup_cl,ct.dropoff_cl,ct.pickup_clon,ct.dropoff_clon,'Distance').take(10)

CPU times: user 41.6 ms, sys: 5.56 ms, total: 47.2 ms
Wall time: 591 ms


[Row(Pickup Centroid Latitude=41.901206994, Dropoff Centroid Latitude=41.901206994, Pickup Centroid Longitude=-87.676355989, Dropoff Centroid Longitude=-87.676355989, Distance=0.0),
 Row(Pickup Centroid Latitude=41.874005383, Dropoff Centroid Latitude=41.9867118, Pickup Centroid Longitude=-87.66351755, Dropoff Centroid Longitude=-87.663416405, Distance=0.11270646238477244),
 Row(Pickup Centroid Latitude=41.899602111, Dropoff Centroid Latitude=41.839086906, Pickup Centroid Longitude=-87.633308037, Dropoff Centroid Longitude=-87.714003807, Distance=0.10086573913913337),
 Row(Pickup Centroid Latitude=41.857183858, Dropoff Centroid Latitude=41.9867118, Pickup Centroid Longitude=-87.620334624, Dropoff Centroid Longitude=-87.663416405, Distance=0.1365046798204703),
 Row(Pickup Centroid Latitude=41.874005383, Dropoff Centroid Latitude=41.857183858, Pickup Centroid Longitude=-87.66351755, Dropoff Centroid Longitude=-87.620334624, Distance=0.046343595040167315),
 Row(Pickup Centroid Latitude=41